# <font color=blue>Project Name : WOE_IV for OneApp Recom</font>

**Notebook**: WoE and IV for OneApp Recommendations<br>

**Title**: It generates WOE_IV.<br>

**Short summary**: It will give use WOE and IV for the all features.<br>

**Analyst**: Aman NAGAR<br>

**Duration** : JULY 2020 <br>

**Version**: 0.5<br>



In [ ]:
%%configure -f
{
"kind": "pyspark",

"conf": {
   
     "spark.master": "yarn",
    "spark.yarn.dist.archives": "/use_case/hhmatching/Library_zip/extra_lib_3.zip#lcm_zip", 
    "spark.yarn.appMasterEnv.PYSPARK_PYTHON": "./lcm_zip/bin/python",
    "spark.driver.cores": "8", 
    "spark.driver.memory": "13000m",  
    "spark.executor.memory": "13000m",
    "spark.executor.cores": "8", 
     "spark.dynamicAllocation.enabled": "true",
     "spark.dynamicAllocation.minExecutors": "3"
    
      
    }
}

### Import Libraries<a name="libs"></a>
<a href = "#toc">Back to top</a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import dataframe
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import lit,log
from pyspark.sql.functions import isnan
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql.functions import col, lit, coalesce, greatest, concat
from pyspark.sql.functions import regexp_replace 
from pyspark.sql import DataFrame
from pyspark.sql.window import *
from pyspark.sql.functions import row_number
from typing import Iterable
import pandas as pd
import numpy as np
from pyspark.sql.types import StringType,FloatType,DoubleType,LongType,IntegerType,ArrayType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Bucketizer
import pandas as pd, numpy as np
from matplotlib import pyplot as plt
#from plotly.offline import iplot
#import plotly.graph_objs as go
#%matplotlib inline
import seaborn as sns
from pandas.plotting import table
#from sklearn.preprocessing import LabelEncoder
import matplotlib.patches as mpatches

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder 
#from sklearn.metrics import roc_curve, auc
#from sklearn.metrics import log_loss
from pyspark.sql import Window
from pyspark.sql.functions import rank,sum,col
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorSlicer

from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.sql.functions import when
from pyspark.sql import functions as F
from pyspark.sql.functions import avg
from pyspark.ml import Pipeline
# Initiate Spark Session
# spark = SparkSession\
#     .builder\
#     .appName("example-spark")\
#     .config("spark.sql.crossJoin.enabled","true")\
#     .getOrCreate()

# ## Creating Hive Context

# sc = pyspark.SparkContext()
# hiveContext = HiveContext(sc)

#spark = SparkSession.builder\
# .master("local[2]")\
# .config("spark.submit.deployMode", "client")\
# .config("spark.eventLog.enabled", "true")\
# .config("spark.dynamicAllocation.enabled", "true")\
# .config("spark.dynamicAllocation.executorIdleTimeout", "1500s")\
# .config("spark.dynamicAllocation.initialExecutors", "53")\
# .appName("Base_Line")\
# .getOrCreate()
#
#sc = spark.sparkContext
#L = spark.sparkContext._conf.getAll()

#bi_reason = sys.argv[0]

#conf = SparkConf().setAppName('Base_Line')
#sc = SparkContext(conf=conf)

#sqlContext = HiveContext(sc)

spark.conf.set("spark.sql.execution.arrow.enabled","false")

spark.conf.set("spark.sql.execution.arrow.fallback.enabled","true")

from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

from pyspark.sql import HiveContext
hiveContext = HiveContext(spark)


### Data_Import_Function

In [ ]:
# Function to load TDM intermediate tables
def dataImport(db,table):
   df = spark.sql("select * from {0}.{1}".format(db,table))
   return df

### Create_Empty_DataFrame

In [ ]:
s = StructType([
            StructField("ticket_fault", StringType(), True)
        ])
di = spark.createDataFrame([],s)

### Pyspark_Qcut_Function

In [ ]:
def pyspark_qcut(pred_var,base):

    base1 = base.withColumn(pred_var,F.col(pred_var).cast(IntegerType()))
    base11 = base1.withColumn(pred_var,base1[pred_var].cast(IntegerType()))
    base11=base11.na.fill(-1234567890)
    base1 = base11.filter(base1[pred_var] != -1234567890)
    base2 = base11.filter(F.col(pred_var) == -1234567890)
    # base2 = base2.withColumn("Range",F.lit("Missing"))
    # base2 = base2.withColumn("Max_R",F.lit(-1234567890))
    # base2 = base2.withColumn("Min_R",F.lit(-1234567890))

    # pred_var = "TotalCharges"

    Variable_range = str(pred_var) + "_Range" 

    result_discretizer = QuantileDiscretizer(numBuckets=10, inputCol=pred_var,outputCol=Variable_range).fit(base1).transform(base1)
    # base1.toPandas()


    vase = result_discretizer.groupBy(Variable_range).agg(F.count(Variable_range).alias("Count"),F.min(pred_var).alias("Min_R"),F.max(pred_var).alias("Max_R"))
   
    # vase.toPandas()

    base1 = result_discretizer.groupBy(Variable_range).agg(F.count(Variable_range).alias("Count"),F.min(pred_var).alias("Min_R"),F.max(pred_var).alias("Max_R"))
    # base1 = base1.withColumn("Min_R",(base1["Min_R"])).withColumn("Max_R",(base1["Max_R"]))

    base11 = result_discretizer.join(base1,Variable_range,"left").drop("count")

    # base11.columns

    base2 = base2.withColumn(Variable_range,F.lit(-1234567890))

    # base2.toPandas()


    base12 = result_discretizer.union(base2)
    
    base1 = base1.select(Variable_range,"Count")
    
    
    # base12.toPandas()
    return base12,base1

### Cross_table_Function

In [ ]:
def cross_j(result_discretizer, pred, ciljna):
    var = result_discretizer.select(pred)
    target = result_discretizer.select(ciljna)
    result_discretizer12 = result_discretizer.crosstab(pred,ciljna)
    
    return result_discretizer12

### WOE&IV_Calculation_Function

In [ ]:
def woe2(result_discretizer12,pred,ciljna):


    n = result_discretizer12.groupBy().sum().collect()[0][0]
    y = result_discretizer12.groupBy().sum().collect()[0][1]
    # y_Percentage = F.udf(lambda z: z/y,FloatType())
    result_discretizer1 = result_discretizer12.withColumn("YES1",(F.col("YES")/y))

    # n_Percentage = F.udf(lambda z: z/n,FloatType())
    result_discretizer1 = result_discretizer1.withColumn("NO1",(F.col("NO")/n))
    result_discretizer2 = result_discretizer1.withColumn("WoE", F.log(F.col("NO1")/F.col("YES1")))
    IV = result_discretizer2.withColumn("IV", F.log(F.col("NO1") / F.col("YES1")) * (F.col("NO1") - F.col("YES1")))

    s = IV.groupBy().sum()

    s1 = s.select("sum(IV)")
    Total_IV = s1.collect()[0][0]

    result_discretizer12 = result_discretizer12.withColumnRenamed('No','Not-repeated')

    result_discretizer12 = result_discretizer12.withColumnRenamed('Yes','Repeated')

    x = result_discretizer12.columns[0]

    y = IV.columns[0]

    IV = IV.withColumnRenamed(y,pred)

    result_discretizer12 = result_discretizer12.withColumnRenamed(x,pred)

    IV = IV.orderBy(pred)
    return IV,Total_IV

### MultiFunction_Merge_Bins

In [ ]:
def Prep(base12,bins_df,var,pred_var):   
#     base12 =F1
#     bins_df = IV
#     Variable_range = F2.columns[0]
#     var = Variable_range
    bins_df1 = bins_df.filter(F.col(var) != -1234567890.0)
    Missing_D1 = bins_df.filter(F.col(var) == -1234567890.0)
    Missing_D11= Missing_D1
    
    bins_df1 = bins_df1.withColumn("dummy",F.lit(1))
    win = Window.partitionBy('dummy').orderBy(F.col(var).asc())
    newDf = bins_df1.select('*', row_number().over(win).alias("row_id"))
    d  = str(var)+"_2"
    Final_D1 = newDf.select(var, "NO","YES",'WoE',"IV",F.lead("WoE",1).over(win).alias("WoE_2"),F.lead("NO",1).over(win).alias("NO_2"),F.lead("YES",1).over(win).alias("YES_2"),F.lead(var,1).over(win).alias(d),row_number().over(win).alias("row_id"))


    Final_D1 = Final_D1.withColumn("Percentage_diff",F.abs((F.col("WoE") - F.col("WoE_2"))/ F.col("WoE")))
    Final_D1 = Final_D1.withColumn("D",F.when(Final_D1.Percentage_diff < 0.10,0).otherwise(1))


    Missing_D1 = Missing_D1.select(var).withColumn("g_id",F.lit(-12345678))

    Final_Dx = Final_D1.filter(F.col("D") ==1 ).select(var,"D")


    win = Window.partitionBy('D').orderBy(F.col("D").asc())
    Final_Dx = Final_Dx.select('*', row_number().over(win).alias("g_id"))

    Final_Dx = Final_Dx.select(var,"g_id")

    Final_D1 = Final_D1.join(Final_Dx, [var] , 'left')

    Final_Dy = Final_D1.select(var,"g_id")




    var_a ="a."+var
    var_b = "b."+var


    Final_Dy1 = Final_Dy.alias('a').crossJoin(Final_Dy.alias('b')) \
        .where((F.col(var_b) >= F.col(var_a)) & (F.col('a.g_id').isNotNull() | F.col('b.g_id').isNotNull())) \
        .withColumn('rn', row_number().over(Window.partitionBy(var_a).orderBy(var_b))) \
        .where(F.col('rn') == 1) \
        .select(var_a, coalesce('a.g_id', 'b.g_id').alias('g_id')) \
        .orderBy(var_a) \
    
    Final_Dy = Final_Dy.select(var)
    Final_Dy = Final_Dy.join(Final_Dy1, [var] , 'left')

    Final_Dy = Final_Dy.na.fill(9)

    FInal = Final_Dy.union(Missing_D1)

    base13 = base12.join(FInal,[var],"left")

    # base13.toPandas()

    # Missing_D1.columns

#     pred_var = "MonthlyCharges"
    base13 = base13.withColumn(pred_var,F.col(pred_var).cast(LongType()))
    base14 = base13.groupBy("g_id").agg(F.count("g_id").alias("Count"),F.min(pred_var).alias("Min_R"),F.max(pred_var).alias("Max_R"))
    # base1 = base1.withColumn("Min_R",(base1["Min_R"])).withColumn("Max_R",(base1["Max_R"]))base1 = base1.withColumn("Min_R",(base1["Min_R"])).withColumn("Max_R",(base1["Max_R"]))


    base13 = base13.drop(var)

    base15 = base14.withColumn(var,F.concat(F.col("Min_R"),F.lit(","),F.col("Max_R")))

    base15 = base15.select("g_id",var)

    base16 = base13.join(base15,"g_id","left")

    base16 = base16.drop("g_id")
    x = base16.count()
    return base16

### Visualization_empty_DataFrame

In [ ]:
g = StructType([
            StructField("Value", StringType(), True),
            StructField("Feature", StringType(), True),
            StructField("NO", IntegerType(), True),
            StructField("YES", IntegerType(), True),
            StructField("YES1", FloatType(), True),
            StructField("NO1", FloatType(), True),
            StructField("WoE", FloatType(), True),
            StructField("IV", FloatType(), True),
            StructField("Main_Feature_IV", FloatType(), True),
            StructField("DATE_OF_ANALYSIS", StringType(), True)
        ])

dg = spark.createDataFrame([],g)
dg = dg.select("NO","YES","Value","Feature","YES1","NO1","WoE","IV","Main_Feature_IV","DATE_OF_ANALYSIS")

### Function_to_calculate_WOE_for _Numeric_features

In [ ]:
def all_WoE_IV(di, df, dg):
    
    #writer = pd.ExcelWriter("all_WoE_test2.xlsx",engine = "xlsxwriter")
    all_WoE = {}
    all_IV = {}
   

   
    
  
        


    for i in set(df.columns) - set(["target", 'ticket_fault' ,"bb_technology"]):
        #'asset_id', 'datum_prijave_smetnje_id', 'datum_prijave_smetnje', 'complaint_process_end_date', ' p2f_date_diff_ss'
        base1 = df.withColumn(i,F.col(i).cast(IntegerType()))
        base11 = base1.withColumn(i,base1[i].cast(IntegerType()))
        base11=base11.na.fill(-1234567890)
        base1 = base11.filter(base1[i] != -1234567890)
        k=base1.count()
        
        if (k > 0):
        
            print("WoE and IV for Variable: {}".format(i))


            F1,F2  = pyspark_qcut(i,df)
            result_discretizer12 = cross_j(F1,F2.columns[0],"target")
            bins_df,bins_IV = woe2(result_discretizer12,F2.columns[0],"target")
            bins_df.show()
            base16  =  Prep(F1,bins_df,F2.columns[0],i)
            result_discretizer12 = cross_j(base16,F2.columns[0],"target")
            bins_df,bins_IV = woe2(result_discretizer12,F2.columns[0],"target")
            IV = bins_df.withColumn('test', regexp_replace(F2.columns[0], ",-", '.'))
            IV = IV.withColumn('test', regexp_replace("test", ",", ''))

            IV = IV.withColumn("test", IV["test"].cast(DoubleType()))

            IV = IV.orderBy("test")
            IV = IV.drop("test")
            bins_df = IV.withColumn(F2.columns[0], regexp_replace(F2.columns[0], '-1234567890,-1234567890', 'Missing'))
#             bins_df = bins_df.withColumn("WoE",-F.col("WoE"))
#             bins_df = bins_df.withColumn("IV",-F.col("IV"))
            bins_df.show()

            bins_df = bins_df.withColumn("Main_Feature_IV",F.lit(bins_IV))
            bins_df = bins_df.withColumn("Feature",F.lit(F2.columns[0]))
            bins_df = bins_df.withColumnRenamed(F2.columns[0],'Value')
            bins_df = bins_df.withColumn("DATE_OF_ANALYSIS", F.date_add(F.current_date(),0))

            bins_df = bins_df.select("NO","YES","Value","Feature","YES1","NO1","WoE","IV","Main_Feature_IV","DATE_OF_ANALYSIS")
            dg = dg.union(bins_df)






            print("Total IV for Variable: {} :{}".format(i,bins_IV))
            print("\n\n")
            base17 = base16.select("ticket_fault",F2.columns[0])
            base17 = base17.withColumn(F2.columns[0], regexp_replace(F2.columns[0], '-1234567890,-1234567890', 'Missing'))
            di = base17.join(di, base17.ticket_fault == di.ticket_fault, 'left').drop( di.ticket_fault)
            all_IV[F2.columns[0]]=bins_IV
        else:
            continue
        
    return di,all_IV,dg

### Function_to_calculate_WOE_for_Categorical_features

In [ ]:
def all_WoE_IV1(di, df,dg):
    
    #writer = pd.ExcelWriter("all_WoE_test2.xlsx",engine = "xlsxwriter")
    all_WoE = {}
    all_IV = {}
    for i in set(df.columns) - set(["target","NPS_submited","NPS_value","business_id", 'ticket_fault',"customer_play_id","oneapp_user_id","date_NPS_submited"]):
        #'asset_id', 'datum_prijave_smetnje_id', 'datum_prijave_smetnje', 'complaint_process_end_date', ' p2f_date_diff_ss'
        
        df=df.na.fill("Missing")
        df = df.withColumn(i, regexp_replace(i, 'null', 'Missing'))
       
#         a2 = a1.filter(a1[i] != "Missing")
#         k=a2.count()
        
        
        print("WoE and IV for Variable: {}".format(i))



        result_discretizer12 = cross_j(df,i,"target")
        bins_df,bins_IV = woe2(result_discretizer12,i,"target")

#         IV = IV.withColumn("test", IV["test"].cast(DoubleType()))

        bins_df = bins_df.orderBy("WoE")


        bins_df.show()

        bins_df = bins_df.withColumn("Main_Feature_IV",F.lit(bins_IV))
        bins_df = bins_df.withColumn("Feature",F.lit(i))
        bins_df = bins_df.withColumnRenamed(i,'Value')
        bins_df = bins_df.withColumn("DATE_OF_ANALYSIS", F.date_add(F.current_date(),0))

        bins_df = bins_df.select("NO","YES","Value","Feature","YES1","NO1","WoE","IV","Main_Feature_IV","DATE_OF_ANALYSIS")
        dg = dg.union(bins_df)
        print("Total IV for Variable: {} :{}".format(i,bins_IV))
        print("\n\n")

        all_IV[i]=bins_IV
        
    return all_IV,dg

### Read_data_from _oneapp_and__churn

In [ ]:

df = dataImport("cdl_common","  dec_31_Fix_Mob_NPS_oib_dec_oa")


In [ ]:
df = df.drop("Rank","NPS_value")

In [ ]:
df = df.withColumn("OneApp_recommended_value",F.col("OneApp_recommended_value").cast(IntegerType()))

In [ ]:
newDf = df.withColumn("target",when(F.col("OneApp_recommended_value") == 0,F.lit("NO")).otherwise(F.lit("YES")))

In [ ]:
newDf = newDf.drop("OneApp_recommended_value","oneapp_user_id")
newDf = newDf.drop("business_id")

In [ ]:
newDf = newDf.withColumnRenamed("oib","ticket_fault")

In [ ]:
len(newDf.columns)

In [ ]:
oneapp_features = ["Live_chat_3m","help_support_6m"]

In [ ]:
ht_features = list(set(newDf.columns )-set(oneapp_features))

In [ ]:
Ht_features_df = newDf.select(*ht_features)

In [ ]:
catcolums_miss=[item[0] for item in Ht_features_df.dtypes if item[1].startswith('string')]  #will select name of column with string data type
numcolumns_miss = [item[0] for item in Ht_features_df.dtypes if item[1].startswith('int') | item[1].startswith('double') | item[1].startswith('bigint') | item[1].startswith('float')] #will select name of column with integer or double data type


In [ ]:
length = list(set(numcolumns_miss)-set( ["oib","target","ticket_fault"]))

In [ ]:
len(length)

In [ ]:
n1 = length[:40]
n2= length[40:80]
n3= length[80:120]
n4 = length[120:160]
n5 = length[160:200]
n6 = length[200:240]
n7 = length[240:280]
n8 = length[280:320]
n9 = length[320:360]
n10 = length[360:]

In [ ]:
df = newDf.select("ticket_fault","target",*n1)

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_n1_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_d1 as select * from myTemp11")

# n2

In [ ]:
df = newDf.select("ticket_fault","target",*n2)

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_n2_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_d2 as select * from myTemp11")

# n3

In [ ]:
df = newDf.select("ticket_fault","target",*n3)

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_n3_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_d3 as select * from myTemp11")

# n4

In [ ]:
df = newDf.select("ticket_fault","target",*n4)

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_n4_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_d4 as select * from myTemp11")

# n5

In [ ]:
df = newDf.select("ticket_fault","target",*n5)

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_n5_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_d5 as select * from myTemp11")

# n6

In [ ]:
df = newDf.select("ticket_fault","target",*n6)

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_n6_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_d6 as select * from myTemp11")

# n7

In [ ]:
df = newDf.select("ticket_fault","target",*n7)

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_n7_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_d7 as select * from myTemp11")

# n8

In [ ]:
df = newDf.select("ticket_fault","target",*n8)

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_n8_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_d8 as select * from myTemp11")

# n9

In [ ]:
df = newDf.select("ticket_fault","target",*n9)

In [ ]:
n9

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:
fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_n9_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_d9 as select * from myTemp11")

# n10

In [ ]:
df = newDf.select("ticket_fault","target",*n10)

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_n10_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_d10 as select * from myTemp11")

# wrong_cat

In [ ]:
wrong_cat = ["AGE_KUPAC",
"BUNDLE_TENURE",
"BB_SP_DAY_TILL_CNTRCT_EXP",
"BNDL_DAY_TILL_CNTRCT_EXP",
"ADSL_TRAFFIC_TENURE",
"ADSL_SPEED_TENURE",
"HALO_TENURE"]

In [ ]:
df = newDf.select("ticket_fault","target",*wrong_cat)

In [ ]:
d1 ,all_IV,fg= all_WoE_IV(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_wrong_cat_oa as select * from myTemp11")


In [ ]:

d1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_4_wrong_cat_d1 as select * from myTemp11")

In [ ]:
catcolums_miss1 = list(set(catcolums_miss) - set(wrong_cat))

In [ ]:
df = newDf.select(*catcolums_miss1)

In [ ]:
all_IV,fg= all_WoE_IV1(di,df,dg)

In [ ]:

fg.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_cat_oa as select * from myTemp11")


In [ ]:

dec_3_n1 = dataImport("cdl_common","  dec_31_n1_oa")

dec_3_n2 = dataImport("cdl_common","  dec_31_n2_oa")
dec_3_n2 = dec_3_n2.select(*dec_3_n1.columns)
dec_3_n3 = dataImport("cdl_common","  dec_31_n3_oa")
dec_3_n3 = dec_3_n3.select(*dec_3_n1.columns)
dec_3_n4 = dataImport("cdl_common","  dec_31_n4_oa")
dec_3_n4 = dec_3_n4.select(*dec_3_n1.columns)

dec_3_n5 = dataImport("cdl_common","  dec_31_n5_oa")
dec_3_n5 = dec_3_n5.select(*dec_3_n1.columns)
dec_3_n6 = dataImport("cdl_common","  dec_31_n6_oa")
dec_3_n6 = dec_3_n6.select(*dec_3_n1.columns)
dec_3_n7 = dataImport("cdl_common","  dec_31_n7_oa")
dec_3_n7 = dec_3_n7.select(*dec_3_n1.columns)
dec_3_n8 = dataImport("cdl_common","  dec_31_n8_oa")
dec_3_n8 = dec_3_n8.select(*dec_3_n1.columns)
dec_3_n9 = dataImport("cdl_common","  dec_31_n9_oa")
dec_3_n9 = dec_3_n9.select(*dec_3_n1.columns)
dec_3_n10 = dataImport("cdl_common","  dec_31_n10_oa")
dec_3_n10 = dec_3_n10.select(*dec_3_n1.columns)
dec_3_wrong_cat = dataImport("cdl_common","  dec_31_wrong_cat_oa")
dec_3_wrong_cat = dec_3_wrong_cat.select(*dec_3_n1.columns)


In [ ]:

dec_3_cat = dataImport("cdl_common","  dec_31_cat_oa")


In [ ]:
dec_3_cat = dec_3_cat.withColumn("dtype",F.lit("Categorical"))

In [ ]:
dec_3_cat = dec_3_cat.filter(F.col("Feature")!="AGE_KUPAC")
dec_3_cat = dec_3_cat.filter(F.col("Feature")!="AGE_KUBUNDLE_TENUREPAC")
dec_3_cat = dec_3_cat.filter(F.col("Feature")!="BB_SP_DAY_TILL_CNTRCT_EXP")
dec_3_cat = dec_3_cat.filter(F.col("Feature")!="BNDL_DAY_TILL_CNTRCT_EXP")
dec_3_cat = dec_3_cat.filter(F.col("Feature")!="ADSL_TRAFFIC_TENURE")
dec_3_cat = dec_3_cat.filter(F.col("Feature")!="ADSL_SPEED_TENURE")
dec_3_cat = dec_3_cat.filter(F.col("Feature")!="HALO_TENURE")

In [ ]:
dec_3 = dec_3_n1.union(dec_3_n2)
dec_3 = dec_3.union(dec_3_n3)
dec_3 = dec_3.union(dec_3_n4)
dec_3 = dec_3.union(dec_3_n5)
dec_3 = dec_3.union(dec_3_n6)
dec_3 = dec_3.union(dec_3_n7)
dec_3 = dec_3.union(dec_3_n8)
dec_3 = dec_3.union(dec_3_n9)
dec_3 = dec_3.union(dec_3_n10)
dec_3 = dec_3.union(dec_3_wrong_cat)
dec_3 = dec_3.withColumn("dtype",F.lit("Numerical"))
dec_3 = dec_3.union(dec_3_cat)

In [ ]:
# dec_3 = dec_3_n1.union(dec_3_n2)
# dec_3 = dec_3.union(dec_3_n3)
# dec_3 = dec_3.union(dec_3_n4)
# dec_3 = dec_3.union(dec_3_n5)
# dec_3 = dec_3.union(dec_3_n6)
# dec_3 = dec_3.union(dec_3_n7)
# dec_3 = dec_3.union(dec_3_n8)
# dec_3 = dec_3.union(dec_3_n9)
# dec_3 = dec_3.union(dec_3_n10)
# dec_3 = dec_3.union(dec_3_wrong_cat)
# # dec_3 = dec_3.union(dec_3_cat)
# dec_3 = dec_3.withColumn("dtype",F.lit("Numeric"))

In [ ]:
dec_3_cat = dec_3_cat.withColumn("dtype",F.lit("Categorical"))

In [ ]:
dec_3 = dec_3.union(dec_3_cat)

In [ ]:
dec_3 = dec_3.withColumn("Total",dec_3.NO + dec_3.YES)

In [ ]:
fg3 = dec_3.withColumnRenamed("YES","detractors")
fg3 = fg3.withColumnRenamed("NO","promotors")
fg3 = fg3.withColumnRenamed("NO1","Distr_promotors")
fg3 = fg3.withColumnRenamed("YES1","Distr_detractors")
fg4 = fg3.select("detractors","promotors","Total","Value","Feature","Distr_promotors","Distr_detractors","WoE","IV","Main_Feature_IV","DATE_OF_ANALYSIS","dtype")
Dec_3_final = fg4.withColumn("DATE_OF_ANALYSIS", F.date_add(F.current_date(),0))

In [ ]:
Dec_3_final = Dec_3_final.filter(F.col("Feature") != "AGE")

In [ ]:
Dec_3_final.count()

In [ ]:

Dec_3_final.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.Dec_311_dec_WOE_IV_oa as select * from myTemp11")
